In [2]:
from __future__ import print_function

import tensorflow as tf
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Conv1D, GlobalMaxPooling1D, Embedding
from keras.datasets import imdb
from keras.utils import plot_model, pad_sequences
from keras import optimizers
import matplotlib.pyplot as plt
import numpy as np
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot

In [3]:
#
# Get data
#
print('Loading data...')
(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=10000)
test_data = x_test
print(len(x_train), 'train sequences')
print(len(x_test), 'test sequences')
#print(x_train[450])

print('Pad sequences (samples x time)')
x_train = tf.keras.preprocessing.sequence.pad_sequences(sequences=x_train, maxlen=800)
x_test =  tf.keras.preprocessing.sequence.pad_sequences(sequences=x_test, maxlen=800)
print('x_train shape:', x_train.shape)
print('x_test shape:', x_test.shape)
print('Build model...')
model = Sequential()

Loading data...
25000 train sequences
25000 test sequences
Pad sequences (samples x time)
x_train shape: (25000, 800)
x_test shape: (25000, 800)
Build model...


In [4]:
#
# prepeare model
#
model.add(Embedding(input_dim=10000, output_dim=100, input_length=800))
model.add(Dropout(0.5))
model.add(Conv1D(filters=250, kernel_size=3, padding='valid', activation='relu', strides=1))
model.add(GlobalMaxPooling1D())
model.add(Dense(250))
model.add(Dropout(0.5))
model.add(Activation('relu'))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])              

In [5]:
model.summary()
#SVG(model_to_dot(model,show_shapes = True).create(prog='dot', format='svg'))

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 800, 100)          1000000   
                                                                 
 dropout (Dropout)           (None, 800, 100)          0         
                                                                 
 conv1d (Conv1D)             (None, 798, 250)          75250     
                                                                 
 global_max_pooling1d (Globa  (None, 250)              0         
 lMaxPooling1D)                                                  
                                                                 
 dense (Dense)               (None, 250)               62750     
                                                                 
 dropout_1 (Dropout)         (None, 250)               0         
                                                        

In [ ]:
#
# Training
#
history = model.fit(x_train, y_train, batch_size=32, epochs=10, validation_data=(x_test, y_test))
history_dict = history.history
history_dict.keys()

In [ ]:
#
# evaluation
#
results = model.evaluate(x_test, y_test)
print ("Accuracy on test set:" , results)
print('Test loss:', results[0])
print('Test accuracy:', results[1])

In [ ]:
#
# Plot
#
val_loss = history.history['val_loss']
loss = history.history['loss']
accuracy = history.history['accuracy']
val_accuracy = history.history['val_accuracy']

epochs = range(1, len(accuracy) + 1)

plt.rcParams['figure.figsize'] = [10, 5]
plt.subplot(1, 2, 1)
plt.plot(epochs, loss, 'bo', label='Training loss', color='red')
plt.plot(epochs,val_loss , 'b', label='Validation loss', color='green')
plt.title('Training and validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(epochs, accuracy, 'bo', label='Training acc', color='red')
plt.plot(epochs, val_accuracy, 'b', label='Validation acc', color='green')
plt.title('Training and validation accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.tight_layout()
plt.show()

In [ ]:
#
# PREDICTION
#
model_prediction = Sequential()
model_prediction.add(Embedding(10000, 50, input_length=800))
model_prediction.add(Dropout(0.5))
model_prediction.add(Conv1D(filters=250, kernel_size=3, padding='valid', activation='relu', strides=1))
model_prediction.add(GlobalMaxPooling1D())
model_prediction.add(Dense(250))
model_prediction.add(Activation('relu'))
model_prediction.add(Dense(1))
model_prediction.add(Activation('sigmoid'))
model_prediction.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy']) 

In [ ]:
#
# training
#
history = model_prediction.fit(x_train, y_train, batch_size=32, epochs=3, validation_data=(x_test, y_test))

In [ ]:
#
# Evaluation
#
results = model.evaluate(x_test, y_test)
print ("Accuracy on test set:" , results)
print('Test loss:', results[0])
print('Test accuracy:', results[1])

In [ ]:
#
# Plot
#
plt.hist(model_prediction.predict(x_test))

In [ ]:
#
# Prediction
#
y_pred = model_prediction.predict(x_test)
prediction_is_positive = y_pred > 0.5
label_is_negative = y_test.reshape((25000,1)) == 0

incorrect_cases = np.where(np.logical_and( prediction_is_positive  , label_is_negative ))[0]
#print ("All incorrect cases: ",incorrect_cases[0:])
print ("Predicted score: ", len(incorrect_cases))